Hay que logearse en la obtener un token

https://developer.nytimes.com/docs/books-product/1/overview

In [1]:
# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd

# Importar librerías para automatización de navegadores web
# -----------------------------------------------------------------------
from time import sleep
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

# Importar librería para hacer solicitudes HTTP
# -----------------------------------------------------------------------
import requests

# Importar librería para suprimir advertencias
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')


# Extracción Datos de la API

In [2]:
def llamar_api(url):
    """
    Realiza una llamada a una API utilizando la URL proporcionada.

    Parameters:
    -----------
    url (str): La URL de la API que se va a llamar.

    Returns:
    --------
    dict or None: Un diccionario con los datos de respuesta de la API si la llamada fue exitosa (código de estado 200).
                  None si la llamada falló o no se pudo autenticar correctamente.
    """
    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.

    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")  # Imprime el código de estado de la respuesta.

    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
        print(f"El motivo por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
    else:
        return llamada.json()  # Si la llamada fue exitosa, devuelve los datos de respuesta en formato JSON.


In [3]:
url = "https://api.nytimes.com/svc/books/v3/reviews.json?author=Stephen+King&api-key=x1KidYoLJF7vUfkX4PFToHAFeGsAswaD"
datos_stephen = llamar_api(url)
datos_stephen.keys()

La llamada a la API nos ha dado una respuesta de tipo: 200


dict_keys(['status', 'copyright', 'num_results', 'results'])

In [4]:
df = pd.DataFrame(datos_stephen["results"])
df.head(2)

,url,publication_dt,byline,book_title,book_author,summary,uuid,uri,isbn13
0,http://www.nytimes.com/2011/11/13/books/review...,2011-11-13,ERROL MORRIS,11/22/63,Stephen King,Stephen King’s time traveler tries to undo som...,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,"[9780307951434, 9780606351461, 9781442344280, ..."
1,http://www.nytimes.com/2011/10/31/books/stephe...,2011-10-31,JANET MASLIN,11/22/63,Stephen King,"Stephen King’s latest novel, “11/22/63,” tells...",00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,"[9780307951434, 9780606351461, 9781442344280, ..."


In [5]:
lista_libros = df["book_title"].unique().tolist()
lista_libros[:4]

['11/22/63',
 'Wolves of the Calla',
 'Nightmares and Dreamscapes',
 'Black House']

# Extracción Datos Selenium

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep

# Crear una instancia del navegador Chrome
driver = webdriver.Chrome()

# Maximizar la ventana del navegador
driver.maximize_window()

# Navegar a la página web de Amazon
driver.get("https://www.bookcenter.es/")

# Esperar 3 segundos para que la página se cargue completamente
sleep(3)

try:
    # Aceptar cookies si el botón está presente
    driver.find_element("css selector", "#cookie-accept").click()
except:
    pass

# Esperar 2 segundos
sleep(2)

# Listas para almacenar datos
precio_disponible = []
toda_la_info = []
libros = []


for libro in lista_libros:
    try:
        # Ingresar el nombre del libro en el cuadro de búsqueda y presionar Enter
        driver.find_element("css selector", "#busqueda > input.search-text").send_keys(libro + " Stephen King's", Keys.ENTER)

        # Esperar 5 segundos para cargar los resultados de búsqueda
        sleep(5)

        # Entrar en la página del libro
        driver.find_element("css selector", "#main-container > div > div > div.row > ul > li:nth-child(1) > div > div > a > img").click()

        # Esperar 2 segundos
        sleep(2)

        # Extraer el precio disponible y agregarlo a la lista
        precio_disponible.append(driver.find_element("css selector", "#main-container > div > div.content.col-md-12 > div > div.price-col.col-md-3.col-sm-3").text)

        # Extraer toda la información y agregarla a la lista
        toda_la_info.append(driver.find_element("css selector", "#main-container > div > div.content.col-md-12 > div > div.fichaDetalle.col-md-9.col-sm-9").text)

        # Agregar el nombre del libro a la lista
        libros.append(libro)

        # Esperar 3 segundos
        sleep(3)

        # Regresar a la página anterior
        driver.back()
    except:
        # Si hay algún error, agregar "Sin datos" a las listas
        precio_disponible.append("Sin datos")
        toda_la_info.append("Sin datos")
        libros.append(libro)

# Cerrar el navegador
driver.close()


## Limpieza de los datos

In [13]:
precio_original = ["Sin datos" if precio == "Sin datos" else float(precio.split()[0].replace(",", ".")) for precio in precio_disponible ]
precio_original[:4]

[23.9, 9.5, 'Sin datos', 'Sin datos']

In [18]:
precio_rebajado = ["Sin datos" if precio == "Sin datos" else float(precio.split("\n")[1].replace(",", ".").replace(" €", "")) for precio in precio_disponible ]
precio_rebajado[:4]

[22.71, 9.03, 'Sin datos', 'Sin datos']

In [26]:
toda_la_info[:2]

['-5%\nHOLLY (EDICIÓN EN ESPAÑOL)\nKING, STEPHEN\nEditorial:\nPLAZA & JANES\nAño de edición:\n2023\nMateria\nNovela terror - suspense\nISBN:\n978-84-01-03111-3\nPáginas:\n624\nEncuadernación:\nTAPA DURA',
 '-5%\nTHE DARK TOWER 5. THE WOLVES OF CALLA\nKING, STEPHEN\nEditorial:\nLIBROS EN INGLES\nAño de edición:\n2006\nISBN:\n978-0-340-82717-8\nPáginas:\n616\nColección:\nLIBROS IMPORTACION SGEL']

In [32]:
editorial = ["Sin datos" if info == "Sin datos" else  info.split("\n")[::2][2].title() for info in toda_la_info]
editorial[:2]

['Plaza & Janes', 'Libros En Ingles']

In [46]:
año = []
for info in toda_la_info:
    try:
        año.append(int(info.split("\n")[::2][3]))
    except:
        año.append("Sin datos")
año[:2]

[2023, 2006]

In [60]:
paginas = []
for info in toda_la_info:
    try: 
        indice_paginas =  info.split("\n").index("Páginas:") + 1
        paginas.append(int(info.split("\n")[indice_paginas]))
    except:
        paginas.append("Sin datos")
paginas[:2]

[624, 616]

# Creación de la BBDD 

```sql
-- -----------------------------------------------------
-- Esquema  libros
-- -----------------------------------------------------
CREATE SCHEMA IF NOT EXISTS `libros` DEFAULT CHARACTER SET utf8 ;
USE `libros` ;

-- -----------------------------------------------------
-- Tabla `libros`.`autores`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `libros`.`autores` (
  `idautores` INT NOT NULL AUTO_INCREMENT,
  `nombre` VARCHAR(100) NOT NULL,
  PRIMARY KEY (`idautores`))
ENGINE = InnoDB;


-- -----------------------------------------------------
-- Tabla `libros`.`editoriales`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `libros`.`editoriales` (
  `ideditoriales` INT NOT NULL AUTO_INCREMENT,
  `nombre_editorial` VARCHAR(100) NOT NULL,
  PRIMARY KEY (`ideditoriales`))
ENGINE = InnoDB;


-- -----------------------------------------------------
-- Tabla `libros`.`títulos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `libros`.`títulos` (
  `idtítulos` INT NOT NULL AUTO_INCREMENT,
  `nombre_libro` VARCHAR(45) NOT NULL,
  `paginas` INT NOT NULL,
  `año_edicion` INT NOT NULL,
  `autores_idautores` INT NOT NULL,
  `editoriales_ideditoriales` INT NOT NULL,
  PRIMARY KEY (`idtítulos`),
  INDEX `fk_títulos_autores1_idx` (`autores_idautores` ASC) VISIBLE,
  INDEX `fk_títulos_editoriales1_idx` (`editoriales_ideditoriales` ASC) VISIBLE,
  CONSTRAINT `fk_títulos_autores1`
    FOREIGN KEY (`autores_idautores`)
    REFERENCES `libros`.`autores` (`idautores`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_títulos_editoriales1`
    FOREIGN KEY (`editoriales_ideditoriales`)
    REFERENCES `libros`.`editoriales` (`ideditoriales`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;

```